In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import randint
import statsmodels.api as sm
from pprint import pprint
import time
import joblib

## Load dataset

In [ ]:
# Load the data
# However, we are unable to share the data set due to the laboratory's confidentiallity protocols.
df = pd.read_csv("Modeling_data_example.csv")

In [ ]:
# Getting columns needed for modeling
cols_livebiomass = ["Livebiomass", "ClGreen", "GNDVI", "MSAVI2", "NDRE", "NDVI", "RVI", "SFDVI", "PreAccmm", "GDD"]
cols_totalbiomass = ["Totalbiomass", "ClGreen", "GNDVI", "MSAVI2", "NDRE", "NDVI", "RVI", "SFDVI", "PreAccmm", "GDD"]

# Live biomass dataframe for modeling
data_livebiomass_df = df[cols_livebiomass]

# Total biomass dataframe for modeling
data_totalbiomass_df = df[cols_totalbiomass]

# Additionally specifying the predictor variables
predictors = ["ClGreen", "GNDVI", "MSAVI2", "NDRE", "NDVI", "RVI", "SFDVI", "PreAccmm", "GDD"]

## Parameter grid for random search

In [ ]:
# Create parameter grid
def create_param_grid():
    """
    Create parameter grid for RandomizedSearchCV
    """
    param_grid = {
        'n_estimators': randint(100, 500),
        'max_depth': randint(10, 30),
        'min_samples_split': randint(2, 10),
        'min_samples_leaf': randint(1, 10),
        'max_features': ['sqrt', 'log2', None]
    }
    return param_grid

param_grid = create_param_grid()

### 1. Live biomass Modeling with Random Forest and RandomizedSearchCV

In [ ]:
Livebiomass = "Livebiomass"

# Prepare data for modeling
X = data_livebiomass_df[predictors].values
y = data_livebiomass_df[Livebiomass].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


In [ ]:
# Find optimal hyperparameters using RandomizedSearchCV
print("Finding optimal hyperparameters...")
start_time = time.time()

# Initialize the base model
rf_base = RandomForestRegressor(random_state=42)

# Initialize RandomizedSearchCV
rf_random = RandomizedSearchCV(
    estimator=rf_base,
    param_distributions=param_grid,
    n_iter=100,  # Number of parameter settings sampled
    cv=5,        # 5-fold cross-validation
    verbose=1,
    random_state=42,
    n_jobs=-1    # Use all available cores
)

# Fit RandomizedSearchCV
rf_random.fit(X_train, y_train)

# Print results
print(f"Best parameters found in {time.time() - start_time:.2f} seconds:")

# Extract best parameters
best_params = rf_random.best_params_
print(best_params)


In [ ]:

# The best parameters found in this stduy for the live biomass model
n_estimators = best_params.get('n_estimators')
max_depth = best_params.get('max_depth')
max_features = best_params.get('max_features')
min_samples_split = best_params.get('min_samples_split')
min_samples_leaf = best_params.get('min_samples_leaf')

In [ ]:
# Create and train the Random Forest model with best parameters
print("\nTraining final model with optimal parameters...")
rf_final = RandomForestRegressor(
    n_estimators=n_estimators,
    max_depth=max_depth,
    max_features=max_features,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf,
    random_state=42
)
rf_final.fit(X_train, y_train)

# Print model parameters
print('Parameters currently in use:\n')
pprint(rf_final.get_params())
print("\n")

# Feature importance analysis
importances = rf_final.feature_importances_
indices = np.argsort(importances)[::-1]  # Sort in descending order

print("Feature importances:", importances)
print("Indices of importance:", indices)

# Display the ranks of the variables
print("Variable Ranks:")
for i, index in enumerate(indices):
    print(f"{i+1}. Variable {predictors[index]}: Importance = {round(importances[index], 3)}")
print("\n")

# Make predictions
All_y_pred = rf_final.predict(X)  # Predictions on entire dataset, this is used for plotting as the maximum value for the x and y axes
Test_y_pred = rf_final.predict(X_test)  # Predictions on test set

# Evaluate the model on the test set
Test_mse = mean_squared_error(y_test, Test_y_pred)
Test_rmse = np.sqrt(Test_mse)
Test_r2 = r2_score(y_test, Test_y_pred)

print("Test RMSE = {:.2f}".format(Test_rmse))
print("Test R² = {:.2f}".format(Test_r2))

# Create scatter plot for test data
plt.figure(figsize=(10, 6))
plt.scatter(y_test, Test_y_pred, edgecolors="black")
max_val = int(np.maximum(y, All_y_pred).max()) + 5
plt.plot([0, max_val], [0, max_val], color='red', linestyle='--')
plt.xlabel('Observed Biomass (g/m²)')
plt.ylabel('Predicted Biomass (g/m²)')
plt.title('Test Set: Observed vs Predicted Biomass')
plt.savefig("Predicted_Livebiomass_test.jpg", dpi=300)
plt.show()

# Statistical analysis using statsmodels
# Add a constant for the intercept
y_with_const = sm.add_constant(y_test)

# Fit an OLS model (Ordinary Least Squares)
model = sm.OLS(Test_y_pred, y_with_const).fit()

# Print summary of the regression model
print(model.summary())

# plot_residuals
residuals = y_test - Test_y_pred
plt.figure(figsize=(8, 5))
plt.scatter(Test_y_pred, residuals, alpha=0.6, color='slateblue')
plt.axhline(y=0, color='red', linestyle='--', linewidth=1)
plt.xlabel("Predicted")
plt.ylabel("Residuals")
plt.title("Predicted_Livebiomass_residuals_test.jpg")
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# Save the pre-trained RF
joblib.dump(rf_final, "./S1_RandomForest_estimation/Livebiomass_RF_model.joblib")

### 2. Total biomass modeling with Random Forest and RandomizedSearchCV

In [ ]:
Totalbiomass = "Totalbiomass"

# Prepare data for modeling
X = data_totalbiomass_df[predictors].values
y = data_totalbiomass_df[Totalbiomass].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


In [ ]:
# Find optimal hyperparameters using RandomizedSearchCV
print("Finding optimal hyperparameters...")
start_time = time.time()

# Initialize the base model
rf_base = RandomForestRegressor(random_state=42)

# Initialize RandomizedSearchCV
rf_random = RandomizedSearchCV(
    estimator=rf_base,
    param_distributions=param_grid,
    n_iter=100,  # Number of parameter settings sampled
    cv=5,        # 5-fold cross-validation
    verbose=1,
    random_state=42,
    n_jobs=-1    # Use all available cores
)

# Fit RandomizedSearchCV
rf_random.fit(X_train, y_train)

# Print results
print(f"Best parameters found in {time.time() - start_time:.2f} seconds:")

# Extract best parameters
best_params = rf_random.best_params_
print(best_params)


In [ ]:
# The best parameters found in this stduy for the total biomass model
n_estimators = best_params.get('n_estimators')
max_depth = best_params.get('max_depth')
max_features = best_params.get('max_features')
min_samples_split = best_params.get('min_samples_split')
min_samples_leaf = best_params.get('min_samples_leaf')

In [ ]:
# Create and train the Random Forest model with best parameters
print("\nTraining final model with optimal parameters...")
rf_final = RandomForestRegressor(
    n_estimators=n_estimators,
    max_depth=max_depth,
    max_features=max_features,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf,
    random_state=42
)
rf_final.fit(X_train, y_train)

# Print model parameters
print('Parameters currently in use:\n')
pprint(rf_final.get_params())
print("\n")

# Feature importance analysis
importances = rf_final.feature_importances_
indices = np.argsort(importances)[::-1]  # Sort in descending order

print("Feature importances:", importances)
print("Indices of importance:", indices)

# Display the ranks of the variables
print("Variable Ranks:")
for i, index in enumerate(indices):
    print(f"{i+1}. Variable {predictors[index]}: Importance = {round(importances[index], 3)}")
print("\n")

# Make predictions
All_y_pred = rf_final.predict(X)  # Predictions on entire dataset, this is used for plotting as the maximum value for the x and y axes
Test_y_pred = rf_final.predict(X_test)  # Predictions on test set

# Evaluate the model on the test set
Test_mse = mean_squared_error(y_test, Test_y_pred)
Test_rmse = np.sqrt(Test_mse)
Test_r2 = r2_score(y_test, Test_y_pred)

print("Test RMSE = {:.2f}".format(Test_rmse))
print("Test R² = {:.2f}".format(Test_r2))

# Create scatter plot for test data
plt.figure(figsize=(10, 6))
plt.scatter(y_test, Test_y_pred, edgecolors="black")
max_val = int(np.maximum(y, All_y_pred).max()) + 5
plt.plot([0, max_val], [0, max_val], color='red', linestyle='--')
plt.xlabel('Observed Biomass (g/m²)')
plt.ylabel('Predicted Biomass (g/m²)')
plt.title('Test Set: Observed vs Predicted Biomass')
plt.savefig("Predicted_Totalbiomass_test.jpg", dpi=300)
plt.show()

# Statistical analysis using statsmodels
# Add a constant for the intercept
y_with_const = sm.add_constant(y_test)

# Fit an OLS model (Ordinary Least Squares)
model = sm.OLS(Test_y_pred, y_with_const).fit()

# Print summary of the regression model
print(model.summary())

# plot_residuals
residuals = y_test - Test_y_pred
plt.figure(figsize=(8, 5))
plt.scatter(Test_y_pred, residuals, alpha=0.6, color='slateblue')
plt.axhline(y=0, color='red', linestyle='--', linewidth=1)
plt.xlabel("Predicted")
plt.ylabel("Residuals")
plt.title("Predicted_Totalbiomass_residuals_test.jpg")
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# Save the pre-trained RF
joblib.dump(rf_final, "./S1_RandomForest_estimation/Totalbiomass_RF_model.joblib")